# PDF table exptraction  

In [ ]:
import os
import sys
import tabula
import PyPDF2
import os
import pandas as pd
import numpy as np

In [ ]:
path = os.getcwd()
path = path+'/../PDFs/'


# Loop over all pdf files in the PDF directory

In [ ]:
for files in os.listdir(path):
    if files.endswith(".pdf"):
        print(files)
    else:
        continue
        

### Reading pdf files using tabula and pypdf 

In [ ]:
# reading PDF file to extract tables from
file = path+"./A.pdf"
tables = tabula.read_pdf(file, multiple_tables=True, pages='all')
pdf_file = PyPDF2.PdfFileReader(open(file,'rb'))

In [ ]:
# get number of pages in pdf file 
totalPages = pdf_file.numPages
print(totalPages)


### First attempt to find and correct 90 deg rotated pages 

In [ ]:
for i in range(1,totalPages):
    #print (pdf_file.getPage(i).get('/Rotate'))
    #check if file is rotated at any angle, if yes, print the information in the file
    if (pdf_file.getPage(i).get('/Rotate') != None and (pdf_file.getPage(i).get('/Rotate')) != 0):
        print("Page: "+str(i)+" "+"Angle: "+str(pdf_file.getPage(i).get('/Rotate')))
        

## cleaning tables 
1. drop based on shape - smaler than 2 in row or column will be droped
2. drop if number of NAN elements are higher than others
3. drop if all columns are objects

In [ ]:
table_clean = []
i=0
j = len(tables)
while i < j:
    #print(i)
    
    if (tables[i-1].shape[1]<2) or (tables[i-1].shape[0]<2):
        #del tables[i-1]
        #j-=1
        i+=1
        continue
    if tables[i-1].isna().sum().sum() > tables[i-1].count().sum():
        
        i+=1
        continue
    
  
    df = pd.DataFrame(tables[(i-1)])
    
    for column in df.columns:
        if (df[column].dtypes != 'object') and (df[column].sum() > df[column].isna().sum()):
            drop = False
            break
        else:
            drop = True
    #print("drop: ", drop)
    if drop:
        #del tables[i-1]
        #j-=1
        i+=1
        continue
    
    #drop columns if all values are NAN
    
    df = df.dropna(axis=1, how='all')
    
    #add to table clean
    table_clean.append(df)
    i+=1
#print(i)


In [ ]:
len(table_clean)

# Fixing "Unnamed" columns 
if half of the coulumn are unnamed, use next row for names (if the next row is str)

In [ ]:
for num in range(len(table_clean)):
    rows_with_nan=[]
    i = 1
    j = table_clean[num].columns.size
    for column in table_clean[num].columns:
        if ("Unnamed"  in column):
            i+=1
    if i>j/2:
        table_clean[num] = table_clean[num].rename(columns=table_clean[num].iloc[0,0:])
        table_clean[num] = table_clean[num].drop([0])
    #re-indexing
    table_clean[num].index = range(len(table_clean[num]))   
    
    # filter rows with nan and add the row index to next row
    rows_with_nan = [index for index, row in table_clean[num].iterrows() if row.isnull().sum() > 2]
    
    if rows_with_nan: 
        for k in rows_with_nan:
            table_clean[num].iat[k+1,0] =  str(table_clean[num].iloc[k,0]) + "-" +  str(table_clean[num].iloc[k+1,0])
        table_clean[num] = table_clean[num].drop(rows_with_nan)
        
    table_clean[num].columns.str.upper()
    
    # if left and bottom of a column is nan move it to left and drop the column
    
    if table_clean[num].columns[0] is np.nan and table_clean[num].columns[1] is not np.nan and table_clean[num].iloc[2,1] is np.nan:
        table_clean[num] = table_clean[num].rename(columns={table_clean[num].columns[0]:str(table_clean[num].columns[1]).upper()})
        table_clean[num] = table_clean[num].drop(columns = [str(table_clean[num].columns[1])])
    
            

In [ ]:
table_clean[4]

In [ ]:
paper_title = pdf_file.getOutlines()[0]["/Title"]


# Saving tables to an exel file

In [ ]:
writer = pd.ExcelWriter('./test2.xlsx', engine='xlsxwriter')
start_row = 1

worksheet_name = "A1"
workbook  = writer.book
worksheet=workbook.add_worksheet(worksheet_name)
writer.sheets[worksheet_name] = worksheet

#worksheet.write_string(0, 0,"Title: " + paper_title)
worksheet.write_string(1, 0,"Table: ")

# Create a format to use in the merged range.
merge_format = workbook.add_format({
    'bold': 1,
    'border': 2,
    'align': 'left',
    'valign': 'vcenter',
    'fg_color': '#00FF00'})

# Merge 5 cells.
worksheet.merge_range('A1:J1',"Title: " + paper_title , merge_format)


for ii in range(len(table_clean)):
    
    #adding cell formatting for table number cells
    Cell_format = workbook.add_format({
    'bold': 1,
    'border': 2,
    'align': 'left',
    'valign': 'vcenter',
    'fg_color': '#FFD7D7'})
    worksheet.write_string(start_row, 0,"Table: {} ".format(ii+1), Cell_format)
    
    
    if ii==0:
        
        table_clean[ii].to_excel(writer, sheet_name=worksheet_name, startrow=start_row, startcol=1, index =False)
    else:
        table_clean[ii].to_excel(writer, sheet_name=worksheet_name, startrow=start_row, startcol=1, index = False)
    
    start_row += table_clean[ii].shape[0]

writer.save()

        
    

# 